En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

### Import Libraries

In [1]:
#! pip install -U memory_profiler

In [3]:
import json
import pandas as pd
import pyspark
import zipfile
from pyspark.sql import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
%load_ext memory_profiler

### Converting zip to gz

In [68]:
%%bash
cd ../data
#unzip -p ../data/farmers-tweets.json.zip | gzip > farmers-tweets.json.gz
unzip -p ../data/farmers-protest-tweets-2021-2-4.json.zip | gzip > farmers-protest-tweets-2021-2-4.json.gz

### Reading tweets as pyspark object

Initially the data is a sample of tweets `./data/farmers-tweets.json`, afterwards we'll load the complete json file

In [74]:
namef = '../data/farmers-protest-tweets-2021-2-4'
#namef = 'farmers-tweets'

file_path = namef+'.json.gz'
print(f"** File to use: {file_path}")

** File to use: ../data/farmers-protest-tweets-2021-2-4.json.gz


The first test is perform the data loading over a RDD sprk object.
`%memit` magic cell allow us measure memory.

*Note: The spark.read.json is running in the .gz file

In [75]:
%memit
#Creating SparkSession 
spark = SparkSession.builder.appName('readJson').getOrCreate()
#Read file as pyspark object()   
data = spark.read.json(file_path)
#transformation and renaming columns steps
dfcol = data.withColumn("created_at", data["date"].cast('date'))\
                                .withColumn("user_id", data["user.id"])\
                                .withColumn("username", data["user.username"])

df = dfcol.select(col("created_at"), col("user_id"), col("username")).groupBy("created_at", "username").count()
df.sort(df["count"].desc()).show(10) 

peak memory: 174.91 MiB, increment: 0.00 MiB


+----------+---------------+-----+
|created_at|       username|count|
+----------+---------------+-----+
|2021-02-19|       Preetm91|  267|
|2021-02-18|neetuanjle_nitu|  195|
|2021-02-17| RaaJVinderkaur|  185|
|2021-02-13|MaanDee08215437|  178|
|2021-02-12|RanbirS00614606|  176|
|2021-02-21|     Surrypuria|  161|
|2021-02-18|  rebelpacifist|  153|
|2021-02-19|KaurDosanjh1979|  138|
|2021-02-23|     Surrypuria|  135|
|2021-02-15|         jot__b|  134|
+----------+---------------+-----+
only showing top 10 rows



### Reading tweets as pandas df

In [76]:
file_path = "../data/farmers-tweets.json"
file_path = '../data/farmers-protest-tweets-2021-2-4.json'


In [43]:
import json
data = [json.loads(line) for line in open(file_path, 'r')]
#print(data[1000:1100])
#data[0]['user']['id']
df = pd.DataFrame(data)

In [46]:
#Read file with json.loads()   
data = [json.loads(line) for line in open(file_path, 'r')]
#convert to dataframe
df = pd.DataFrame(data)
#transformation and renaming columns steps
df["created_at"] = pd.to_datetime(df["date"]).dt.strftime('%Y-%m-%d')
#lambda functions to create new columns from user tag
df['username'] = df['user'].apply(lambda d: d['username'])
df['user_id'] = df['user'].apply(lambda d: d['id'])

dfres = df.groupby(['created_at','username'])['id'].count().reset_index(name="count").sort_values("count", ascending=False) 

print(dfres.head(10))

      created_at         username  count
1981  2021-02-24  MaanDee08215437     57
1393  2021-02-20  MangalJ23056160     57
2294  2021-02-24   preetysaini321     43
261   2021-02-15   Rajesh09Sangam     36
713   2021-02-17      Gurpreetd86     32
1098  2021-02-17   preetysaini321     30
13    2021-02-15      AhluwaliaA2     27
368   2021-02-15    akshithepatel     25
1085  2021-02-17       nonymous_i     23
310   2021-02-15        SonaG2022     22


In [8]:
df["created_at"] = pd.to_datetime(df["date"]).dt.strftime('%Y-%m-%d')